## This project is aimed at using an automation module "Selenium" to scrape performance data of footballers in English Premier League for the 2022/2023 season from a football data site "fbref.com". Three steps were involved in the entire operation.
- ### Extraction
- ### Transformation
- ### Loading


##### EXTRACTION STEPS
- ##### The primary tool here is Selenium, which interact with the web page using a driver. for the purpose of this project, a chrome driver matching the desktop windows specification was downloaded and added to project folder.
- ##### Creating an object of the driver and web URL.
- ##### Inspecting web page for static build.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Provide the path to the downloaded driver executable
driver_path = "chromedriver.exe"

# Create a new instance of the web driver
driver = webdriver.Chrome(executable_path=driver_path)  # Replace with the appropriate driver

url = "https://fbref.com/en/comps/9/stats/Premier-League-Stats"
driver.get(url)

#table_XPAth = //*[@id="stats_standard"]/tbody/tr[1]/td[1]
#header_XPath = //*[@id="stats_standard"]/thead/tr[2]/th
#remove row position locator from table_XPath element
#table_XPAth = //*[@id="stats_standard"]/tbody/tr/td


head = [] #contains column titles
for header in driver.find_elements(By.XPATH, '//*[@id="stats_standard"]/thead/tr[2]/th'):
    head.append(header.text)

table = [] #contains data
for names in driver.find_elements(By.XPATH, '//*[@id="stats_standard"]/tbody/tr/td'):
    table.append(names.text)


C:\Users\HP\AppData\Local\Temp\ipykernel_49084\922639451.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)  # Replace with the appropriate driver


##### TRANSFORMATION STEPS
- ##### Removing of redundant columns
- ##### Renaming of Columns
- ##### Creation of individual lists
- ##### Loading to a pandas Dataframe
- ##### Data Type Casting

In [2]:
#remove first element as it is not useful to final table
head.pop(0)

# change the name of elment "Gls" as it is duplicated
head[25] = 'Gls/90'
head[26] = 'ASt/90'
head[27] = 'G+A/90'
head[28] = 'G-PK/90'
head[29] = 'G+A-PK/90'
head[30] = 'XG/90'
head[31] = 'XAG/90'
head[32] = 'XG+XAG/90'
head[33] = 'npXG/90'
head[34] = 'npXG+XAG/90'


In [3]:
def splitter(lst, size):
    '''this function splits a list into lists of designated element size
    lst = list to be splitted
    size = length of each list'''
    for i in range(0, len(lst), size):
        x =  lst[i:i+size]
        yield x

In [4]:
# using the function to split the table list
table_data = list(splitter(table, 36))

In [5]:
#creating a pandas datframe for table_data
import pandas as pd
data = pd.DataFrame(table_data, columns=head)
data.drop('Matches', axis=1, inplace=True)

In [6]:
data.columns

Index(['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts',
       'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY',
       'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR',
       'Gls/90', 'ASt/90', 'G+A/90', 'G-PK/90', 'G+A-PK/90', 'XG/90', 'XAG/90',
       'XG+XAG/90', 'npXG/90', 'npXG+XAG/90'],
      dtype='object')

In [7]:
# Removes comma from "Min" column values
data['Min'] = data["Min"].str.replace(",", "")


In [8]:
data[["Age", "Born", "MP", "Starts", "Min", "Gls", "Ast", "G+A", 
      "PK", "G-PK", "PKatt", "CrdY", "CrdR", "PrgC", "PrgP", "PrgR"]] = data[["Age", "Born", "MP", "Starts", "Min", "Gls", "Ast", "G+A", 
      "PK", "G-PK", "PKatt", "CrdY", "CrdR", "PrgC", "PrgP", "PrgR"]].astype(int)

In [9]:
data[['Gls/90', 'ASt/90', 'G+A/90', 'G-PK/90', 'G+A-PK/90', 'XG/90', 'XAG/90',
       'XG+XAG/90', 'npXG/90', 'npXG+XAG/90', 'xG', 'npxG', 'xAG', 'npxG+xAG', '90s']] = data[['Gls/90', 'ASt/90', 'G+A/90', 'G-PK/90', 'G+A-PK/90', 'XG/90', 'XAG/90',
       'XG+XAG/90', 'npXG/90', 'npXG+XAG/90', 'xG', 'npxG', 'xAG', 'npxG+xAG', '90s']].astype(float)

##### LOADING STEP
- ##### Loading of tabular data to MySQL Datatbase

In [20]:
import pandas as pd
from sqlalchemy import create_engine
import configparser



config = configparser.ConfigParser()
config.read('config.ini')

# declare variables for config values
db_host = config['inputs']['host']
db_user = config['inputs']['user']
db_password = config['inputs']['password']
db_name = config['inputs']['db']
db_port = config['inputs']['port']

# MySQL database connection details
# db_host = config.host
# db_port = config.por = t
# db_user = config.user
# db_password = config.password
# db_name = config.database


# Create the MySQL engine
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Upload the DataFrame to the MySQL database table
data.to_sql('epl_player_stats', con=engine, if_exists='replace', index=False)

# Close the database connection
engine.dispose()


In [ ]:
for nn in range(1,200):
for hh in range(1,20):
    if hh%nn == 0:
        print(hh)
    else:
        continue
nn +=1
